# Openai Proxy Quickstart
> A simple tool to capture metadata about your interactions with the OpenAI API.  Currently supports ChatCompletion endpoint, including streaming.

## Setup

All you need to do is:
1. Change your openai api key to the format `{wandb_api_key}:{openai_api_key}`; and
2. Change your openai api base url to `https://wandb.ai/proxy/openai/v1`

In [ ]:
import os
import openai

openai_api_key = os.getenv("OPENAI_API_KEY")
wandb_api_key = os.getenv("WANDB_API_KEY")

In [ ]:
openai.api_base = "https://api.wandb.ai/proxy/openai/v1"
openai.api_key = f"{wandb_api_key}:{openai_api_key}"

## Start logging
You can use your existing API calls as-is, now with extra logging!

By default, this will log to `{your_entity}/proxy/openai`, where the project is `proxy` and the table name is `openai`.


### Using OpenAI SDK

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=256,
    messages=[{"role": "user", "content": "Tell me a joke about gradients!"}],
)
response

### Using CURL

In [ ]:
%%bash
curl "https://api.wandb.ai/proxy/openai/v1/chat/completions" \
-H "Authorization: Bearer $WANDB_API_KEY:$OPENAI_API_KEY" \
-H "Content-Type: application/json" \
-d '{
     "model": "gpt-3.5-turbo",
     "messages": [{"role": "user", "content": "Tell me a joke about loss functions!"}],
     "temperature": 0.7
   }'

## Configuration
Optionally specify headers to determine where you want your data to be created.

| Header | Description | Example | Default |
| --- | --- | --- | --- |
| `X-Wandb-Entity` | W&B entity name | `wandb` | Your default entity |
| `X-Wandb-Project` | W&B project name | `examples` | `monitoring` |
| `X-Wandb-Stream` | StreamTable name | `openai-chat-test` | `openai` |

By default, proxy data will show up in `{your_entity}/proxy/openai`.

### Advanced configuration
1. Optionally add custom attributes to your table with the `X-Wandb-Attribute-` prefix.  For example, adding the header `X-Wandb-Attribute-Foo: bar` will add a column `Foo` with the value `bar` to your table.
2. Optionally add the `X-Wandb-Client-Id` header if you want to further group related requests together.  By default, each request will have its own unique Client ID.

### Using OpenAI SDK

In [ ]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=256,
    messages=[{"role": "user", "content": "Tell me a joke about gradients as if you were Shakespeare"}],
    headers={
        "X-Wandb-Entity": "andrew",
        "X-Wandb-Project": "examples",
        "X-Wandb-Stream": "openai-chat-test",
        "X-Wandb-Client-Id": "Shakespearean-Gradients-001",
        "X-Wandb-Attribute-Persona": "Shakespeare",
    }
)
response

### Use with CURL
Note: We use the `X-OpenAI-Client-User-Agent` header to determine that your request is intended for OpenAI.  You can use a valid user agent, or just `{}` as shown below.

In [ ]:
%%bash
curl "https://api.wandb.ai/proxy/openai/v1/chat/completions" \
-H "Authorization: Bearer $WANDB_API_KEY:$OPENAI_API_KEY" \
-H "X-Wandb-Entity: wandb" \
-H "X-Wandb-Project: examples" \
-H "X-Wandb-Stream: openai-chat-test" \
-H "X-Wandb-Client-Id: Shakespearean-Gradients-001" \
-H "X-Wandb-Attribute-Persona: Shakespeare" \
-H "Content-Type: application/json" \
-d '{
     "model": "gpt-3.5-turbo",
     "messages": [{"role": "user", "content": "Tell me a joke about loss functions!"}],
     "temperature": 0.7
   }'